# AutoGrad

当我们训练神经网络算法的时候，我们通常要用于梯度的反向传播(back propagation)。在反向传播算法中，我们神经网络的参数会根据它们对于最终的损失函数的梯度来调整。

$$\text{output} = f(\text{input}, W)$$
$$\text{Loss} = L(\text{output}, \text{groudtruth}) =  L(f(\text{input}, W), \text{groudtruth})$$
$$W = W - \eta * W_g$$

为了计算公式中的损失函数$L$对于参数$W$的梯度$W_g$，Pytorch提供了一个很强大的自动求梯度的引擎：`torch.autograd`。它支持对于任何计算图的自动梯度计算。

下面我们演示了，如果利用torch.autograd，来求于一个简单的函数的反向求梯度的过程，其中`w`是参数，最终求取的是loss对w的梯度。

In [46]:
import torch

x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

In [47]:
loss.backward()
print(f'grad of w is {w.grad}')

grad of w is tensor([[0.0180, 0.1190, 0.0868],
        [0.0180, 0.1190, 0.0868],
        [0.0180, 0.1190, 0.0868],
        [0.0180, 0.1190, 0.0868],
        [0.0180, 0.1190, 0.0868]])


在上面的例子中，因为我们想要求取`w`的梯度，所以在创建`w`的时候，设置了`requires_grad=True`，表明我们后续要求取`w`的梯度。

我们可以在创建的时候指定`requires_grad=True`，也可以在创建后，通过`w.requires_grad_(True)`来修改。

# 计算图

上面的计算过程，我们可以用下面的计算图来表示：

<img src='./images/comp-graph.png' width=500>

在这个示例网络计算中,w和b是parameters，是我们需要更新优化的部分。

我们沿着计算图正方向计算，就可以计算出loss的值 ，而反向传播就是把梯度沿着计算图反方向计算。

下面可以通过一个pytorch中一个标准的计算图计算函数，来大概看出来pytorch是怎么实现正向传播与反向传播的。

```python
class LinearFunction(Function):

    # Note that both forward and backward are @staticmethods
    @staticmethod
    # bias is an optional argument
    def forward(ctx, input, weight, bias=None):
        ctx.save_for_backward(input, weight, bias)
        output = input.mm(weight.t())
        if bias is not None:
            output += bias.unsqueeze(0).expand_as(output)
        return output

    # This function has only a single output, so it gets only one gradient
    @staticmethod
    def backward(ctx, grad_output):
        # This is a pattern that is very convenient - at the top of backward
        # unpack saved_tensors and initialize all gradients w.r.t. inputs to
        # None. Thanks to the fact that additional trailing Nones are
        # ignored, the return statement is simple even when the function has
        # optional inputs.
        input, weight, bias = ctx.saved_tensors
        grad_input = grad_weight = grad_bias = None

        # These needs_input_grad checks are optional and there only to
        # improve efficiency. If you want to make your code simpler, you can
        # skip them. Returning gradients for inputs that don't require it is
        # not an error.
        if ctx.needs_input_grad[0]:
            grad_input = grad_output.mm(weight)
        if ctx.needs_input_grad[1]:
            grad_weight = grad_output.t().mm(input)
        if bias is not None and ctx.needs_input_grad[2]:
            grad_bias = grad_output.sum(0)

        return grad_input, grad_weight, grad_bias
```

核心就是正向计算时，通过ctx把反向时，需要记录的Tenosr，都保存起来，反向时，再取出来用。

# 停止梯度的跟踪

在有些时间，我们可能会对我们参数进行一些别的运算，这部分的运算并不是训练的一部分，不需要记录在整个计算图中，进行梯度跟踪。

这时候我们可以用`torch.no_grad()` 块作用域来达到目的。比如在上面的简单的网络示例中，如果我们只是想对我们训练好的模型(w,b)，做一次testing。

In [48]:
print(z.requires_grad)
with torch.no_grad():
    z = torch.matmul(x, w) + b
print(z.requires_grad)

True
False


除此之外，我们还可以使用`detach()`方法，detatch返回的Tensor和原Tensor数据是相同的，但对返回Tensor的任何计算，都不会统计到计算图中。

In [49]:
z = torch.matmul(x, w.detach()) + b.detach()
print(z.requires_grad)

False


但是如果我们对detach后返回的Tenosr进行了修改，那原Tensor也会被对应修改，这时候就破坏了原有的计算图，就会报错。

```text
RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor [1, 5]] is at version 1; expected version 0 instead. Hint: enable anomaly detection to find the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True).
```

In [53]:
print(f'x : {x}')
z = torch.matmul(x,w) + b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)
# 在形成计算图后，对原计算图中的一个，被用于grad_fn输入的Tensor: x，进行了修改
x1 = x.detach()
x1[0] = 2
print(f'x : {x}')
# loss.backward()

x : tensor([2., 1., 1., 1., 1.])
x : tensor([2., 1., 1., 1., 1.])


一旦backward被调用，这个计算图就被释放了，除非指定backward的参数`retain_graph`，这就可以让我们使用起来更加灵活。

我们可以在每一次forward和backward后，改变网络的结构（DAG的样子），每次forward时会重新构建新的计算图。